In [125]:
import os
import numpy as np
import cv2
import mediapipe as mp
import math

In [126]:
images_dir = "./Training/"
file_list = os.listdir(images_dir)
file_list.sort(key=lambda a:int(a[:-4]))

In [127]:
primer = cv2.imread(images_dir + file_list[1])
train_image = cv2.imread(images_dir + file_list[3])
cv2.imwrite("/Users/ivandemyanov/Desktop/primer.jpg", primer)
cv2.imwrite("/Users/ivandemyanov/Desktop/tr.jpg", train_image)

True

### Определение точек, образующих линию пальцев

In [303]:
def get_handList_and_image(image_path):
    
    image = cv2.imread(image_path)
    mp_Hands = mp.solutions.hands
    hands = mp_Hands.Hands()
    
    RGB_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(RGB_image)
    multiLandMarks = results.multi_hand_landmarks
    if multiLandMarks:
        handList = []
        for handLms in multiLandMarks:
            for idx, lm in enumerate(handLms.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                handList.append((cx, cy))
    if multiLandMarks:
        handList = []
        for handLms in multiLandMarks:
            for idx, lm in enumerate(handLms.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                handList.append((cx, cy))
    return handList, image

In [304]:
def make_actual_dots_list(dots_list):
    actual_dots_list = []
    actual_dots_list.append(dots_list[4])
    actual_dots_list.append(((dots_list[1][0] + dots_list[5][0]) // 2, (dots_list[1][1] + dots_list[5][1]) // 2))
    actual_dots_list.append(dots_list[8])
    actual_dots_list.append(((dots_list[5][0] + dots_list[9][0]) // 2, (dots_list[5][1] + dots_list[9][1]) // 2))
    actual_dots_list.append(dots_list[12])
    actual_dots_list.append(((dots_list[9][0] + dots_list[13][0]) // 2, (dots_list[9][1] + dots_list[13][1]) // 2))
    actual_dots_list.append(dots_list[16])
    actual_dots_list.append(((dots_list[13][0] + dots_list[17][0]) // 2, (dots_list[13][1] + dots_list[17][1]) // 2))
    actual_dots_list.append(dots_list[20])
    return actual_dots_list

In [305]:
def draw_circles(image, hand_lst):
    for dot in hand_lst:
        center_coordinates = dot
        radius, color = 5, (255, 0, 0)
        thickness = 2
        image = cv2.circle(image, center_coordinates, radius, color, thickness)
    return image

In [306]:
def draw_lines(image, hand_lst):
    for i in range(8):
        start_point = hand_lst[i]
        end_point = hand_lst[i+1]
        color = (0, 255, 0)
        thickness = 2
        image = cv2.line(image, start_point, end_point, color, thickness)
    return image

In [316]:
handList, img = get_handList_and_image(images_dir + file_list[15])
act_handList = make_actual_dots_list(handList)
img = draw_circles(img, act_handList)
img = draw_lines(img, act_handList)

In [317]:
cv2.imwrite("/Users/ivandemyanov/Desktop/val14.jpg", img)

True

### Определение позы ладони

In [330]:
image = cv2.imread("./Training/" + file_list[11])
cv2.imwrite("/Users/ivandemyanov/Desktop/val24.jpg", image)

True

In [328]:
def make_vectors(a, b, c):
    return (a[0] - b[0], a[1] - b[1]), (c[0] - b[0], c[1] - b[1])

def pose_definition(act_hand_List):
    
    p12 = make_vectors(act_hand_List[0], act_hand_List[1], act_hand_List[2])
    p23 = make_vectors(act_hand_List[2], act_hand_List[3], act_hand_List[4])
    p34 = make_vectors(act_hand_List[4], act_hand_List[5], act_hand_List[6])
    p45 = make_vectors(act_hand_List[6], act_hand_List[7], act_hand_List[8])
    
    return '1'+big_finger_pose_predict(p12[0], p12[1])+'2'+pose_predict(p23[0], p23[1])+'3'+\
        pose_predict(p34[0], p34[1])+'4'+pose_predict(p45[0], p45[1])+'5'

def find_angle(a, b):
    m1, m2 = np.sqrt(a[0]**2 + a[1]**2), np.sqrt(b[0]**2 + b[1]**2)
    angl = np.arccos((a[0]*b[0] + a[1]*b[1]) / (m1 * m2))
    return angl
    
def big_finger_pose_predict(a, b):
    angl = find_angle(a, b)
    if angl < (math.pi / 180) * 39:
        return '+'
    return '-'

def pose_predict(a, b):
    angl = find_angle(a, b)
    if angl < (math.pi / 180) * 25:
        return '+'
    return '-'

In [331]:
handList, img = get_handList_and_image(images_dir + file_list[11])
act_handList = make_actual_dots_list(handList)
pose_definition(act_handList)

'1-2-3-4+5'

### Запись результатов в файл

In [335]:
f = open('Results.txt', 'w')
for file in file_list:
    handList, img = get_handList_and_image(images_dir + file)
    act_handList = make_actual_dots_list(handList)
    f.write(pose_definition(act_handList) + '\n')
    string = ('!,'+file+',T '+str(act_handList[0][0])+' '+str(act_handList[0][1])+',T '+
         str(act_handList[2][0])+' '+str(act_handList[2][1])+',T '+
         str(act_handList[4][0])+' '+str(act_handList[4][1])+',T '+
         str(act_handList[6][0])+' '+str(act_handList[6][1])+',T '+
         str(act_handList[8][0])+' '+str(act_handList[8][1])+',V '+
         str(act_handList[1][0])+' '+str(act_handList[1][1])+',V '+
         str(act_handList[3][0])+' '+str(act_handList[3][1])+',V '+
         str(act_handList[5][0])+' '+str(act_handList[5][1])+',V '+
         str(act_handList[7][0])+' '+str(act_handList[7][1])+',?')
    f.write(string + '\n')
f.close()